In [0]:
!pip install gym-retroº

    100% |████████████████████████████████| 158.7MB 211kB/s 


In [3]:
! wget http://www.atarimania.com/roms/Roms.rar && unrar x Roms.rar && unzip Roms/ROMS.zip

/bin/sh: wget: command not found


In [0]:
! ls

Roms  ROMS  Roms.rar  sample_data


In [1]:
! python3 -m retro.import ROMS/

Importing 1904 potential games...
Importing Alien-Atari2600
Importing CrazyClimber-Atari2600
Importing Tutankham-Atari2600
Importing Tennis-Atari2600
Importing Amidar-Atari2600
Importing NameThisGame-Atari2600
Importing Venture-Atari2600
Importing Assault-Atari2600
Importing Gopher-Atari2600
Importing Atlantis-Atari2600
Importing Pong-Atari2600
Importing VideoPinball-Atari2600
Importing Assault-Atari2600
Importing AirRaid-Atari2600
Importing Qbert-Atari2600
Importing Robotank-Atari2600
Importing Hero-Atari2600
Importing WizardOfWor-Atari2600
Importing StarGunner-Atari2600
Importing Qbert-Atari2600
Importing Jamesbond-Atari2600
Importing YarsRevenge-Atari2600
Importing RoadRunner-Atari2600
Importing MontezumaRevenge-Atari2600
Importing Assault-Atari2600
Importing Krull-Atari2600
Importing Centipede-Atari2600
Importing Skiing-Atari2600
Importing UpNDown-Atari2600
Importing Pitfall-Atari2600
Importing MsPacman-Atari2600
Importing DoubleDunk-Atari2600
Importing Enduro-Atari2600
Importing B

### Confirm GPU

In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: GPU device not found

In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

### Creating the environment

In [ ]:
import tensorflow as tf
import numpy as np
import retro #Retro environment

from skimage import transform #Preprocess the frames
from skimage.color import rgb2gray #To gray the frames

import matplotlib.pyplot as plt #Display graphs

from collections import deque #Ordered collections with ends

import random

import warnings #To ignore all warning messages printed during training
warnings.filterwarnings('ignore')

In [5]:
#Riverraid-Atari2600
#Hero
#Pitfall

#env = retro.make(game='SpaceInvaders-Atari2600')

print("Size of frame:", env.observation_space)
print("Actions size:", env.action_space.n)

#Create a hot encoded version of our actions
#possible_actions = [[1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], ...]
possible_actions = np.array(np.identity(env.action_space.n, dtype=int).tolist())
#For space invaders just actions 6,7 and 1 matters
actions = []
actions.append(possible_actions[0])
actions.append(possible_actions[6])
actions.append(possible_actions[7])

#possible_actions = np.array(actions)[:,[0,6,7]]
possible_actions = np.array(actions)

print(possible_actions[2])

Size of frame: Box(210, 160, 3)
Actions size: 8
[0 0 0 0 0 0 0 1]


## Preprocess frames


In [6]:
def preprocess_frame(frame):
  gray = rgb2gray(frame)
  
  #Crop the screen  [Up: Down, Left: Rigth]
  cropped_frame = gray[8:-12, 4:-12]

  #Normalize
  normalized_frame = cropped_frame/255.0
  
  #Resize
  prepropcessed_frame = transform.resize(cropped_frame, [110,84])
  
  return prepropcessed_frame #110x84x1

## Stack the frames

In [7]:
stack_size = 4 # We stack 4 frames

# Initialize deque with zero-images one array for each image
stacked_frames  =  deque([np.zeros((110,84), dtype=np.int) for i in range(stack_size)], maxlen=4)

def stack_frames(stacked_frames, state, is_new_episode):
    # Preprocess frame
    frame = preprocess_frame(state)
    
    if is_new_episode:
        # Clear our stacked_frames
        stacked_frames = deque([np.zeros((110,84), dtype=np.int) for i in range(stack_size)], maxlen=4)
        
        # Because we're in a new episode, copy the same frame 4x
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        
        # Stack the frames
        stacked_state = np.stack(stacked_frames, axis=2)
        
    else:
        # Append frame to deque, automatically removes the oldest frame
        stacked_frames.append(frame)

        # Build the stacked state (first dimension specifies different frames)
        stacked_state = np.stack(stacked_frames, axis=2) 
    
    return stacked_state, stacked_frames

## Hiperparameters

In [8]:
### MODEL HYPERPARAMETERS
state_size = [110, 84, 4]      # Our input is a stack of 4 frames hence 110x84x4 (Width, height, channels) 
action_size = env.action_space.n # 3 possible actions
learning_rate =  0.00025      # Alpha (aka learning rate)

### TRAINING HYPERPARAMETERS
total_episodes = 15            # Total episodes for training
max_steps = 50000              # Max possible steps in an episode
batch_size = 64                # Batch size

# Exploration parameters for epsilon greedy strategy
explore_start = 1.0            # exploration probability at start
explore_stop = 0.10            # minimum exploration probability 
decay_rate = 0.000012          # exponential decay rate for exploration prob

# Q learning hyperparameters
gamma = 0.9                    # Discounting rate

### MEMORY HYPERPARAMETERS
pretrain_length = batch_size   # Number of experiences stored in the Memory when initialized for the first time
memory_size = 1000000          # Number of experiences the Memory can keep

### PREPROCESSING HYPERPARAMETERS
stack_size = 4                 # Number of frames stacked

### MODIFY THIS TO FALSE IF YOU JUST WANT TO SEE THE TRAINED AGENT
training = False

## TURN THIS TO TRUE IF YOU WANT TO RENDER THE ENVIRONMENT
episode_render = False

### Create Deep Q-Learning Neural Network Model



*   Stack de 4 frames como input
*   Se pasa por 3 convnets
*   Luego se flatened 
*   Finalmente se pasa por 2 FC layers (Fully connected layers)
*   Output o sale el Q value de cada accion





In [9]:
class DQNetwork():
  def __init__(self, state_size, action_size, learning_rate, name='DQNetwork'):
    self.state_size = state_size
    self.action_size = action_size
    self.learning_rate = learning_rate
    
    with tf.variable_scope(name):
      #placeholders
      #*state_size means that we take each element of the state_size in tuple hence is like if we wrote
      #[None, 84, 84, 4]
      
      self.inputs_ = tf.placeholder(tf.float32, [None, *state_size], name="inputs")
      self.actions_ = tf.placeholder(tf.float32, [None, self.action_size], name="actions_")
      
      #target_q is the R(s,a) + ymax Qhat(s',a') : estimated Q value
      self.target_Q = tf.placeholder(tf.float32, [None], name="target")
      
      #First convnet:
      #CNN
      #ELU
      
      #Input is 110x84x4
      self.conv1 = tf.layers.conv2d(inputs = self.inputs_,
                                   filters = 32,
                                   kernel_size = [8,8],
                                   strides = [4,4],
                                   padding = 'VALID',
                                   kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d(),
                                   name = 'conv1'
                                   )
      
      self.conv1_out = tf.nn.elu(self.conv1, name='conv1_out')
      
      #Second convnet:
      #CNN
      #ELU
      
      self.conv2 = tf.layers.conv2d(inputs = self.conv1_out,
                                   filters = 64,
                                   kernel_size = [4,4],
                                   strides = [2,2],
                                   padding = 'VALID',
                                   kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d(),
                                   name = 'conv2'
                                   )
      
      self.conv2_out = tf.nn.elu(self.conv2, name='conv2_out')
      
      #Third convnet:
      #CNN
      #ELU
      
      self.conv3 = tf.layers.conv2d(inputs = self.conv2_out,
                                   filters = 64,
                                   kernel_size = [3,3],
                                   strides = [2,2],
                                   padding = 'VALID',
                                   kernel_initializer = tf.contrib.layers.xavier_initializer_conv2d(),
                                   name = 'conv3'
                                   )
      
      self.conv3_out = tf.nn.elu(self.conv3, name='conv3_out')
      
      #Flatten
      self.flatten = tf.contrib.layers.flatten(self.conv3_out)
      
      self.fc = tf.layers.dense(inputs = self.flatten,
                                units = 512,
                                activation = tf.nn.elu,
                                kernel_initializer = tf.contrib.layers.xavier_initializer(),
                                name = 'fc1'
                                )
      
      #Output the Q value for each action
      self.output = tf.layers.dense(inputs = self.fc,
                                   kernel_initializer = tf.contrib.layers.xavier_initializer(),
                                  units = self.action_size,
                                  activation = None
                                  )
      
      #Q is our predicted value
      self.Q = tf.reduce_sum(tf.multiply(self.output, self.actions_))
      print('output: ', self.output)
      print('actions_: ', self.actions_)
      
      #The loss is the difference between our predicted Q_values and the Q_target
      #Sum(Q_target - Q)^2
      self.loss = tf.reduce_mean(tf.square(self.target_Q - self.Q))
      
      
      #Optimize using AdamOptimizer tho minimize the loss 
      #For training the Network
      self.optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)
  

In [10]:
#Reset the graph
tf.reset_default_graph()

#Instantiate the DQNetwork
DQNetwork = DQNetwork(state_size, action_size, learning_rate)

output:  Tensor("DQNetwork/dense/BiasAdd:0", shape=(?, 8), dtype=float32)
actions_:  Tensor("DQNetwork/actions_:0", shape=(?, 8), dtype=float32)


### Experience Replay

Implement the Experience replay method
We create the Memory object that creates a deque.A deque (double ended queue) **is a data type that removes the oldest element each time that a new one is added** . \n
Code from Udacity

In [11]:
class Memory():
    def __init__(self, max_size):
        self.buffer = deque(maxlen = max_size)
   
    #add new experience to the deque
    def add(self, experience):
        self.buffer.append(experience)
    
    #Create a batch of randomize experience
    def sample(self, batch_size):
        buffer_size = len(self.buffer)
        index = np.random.choice(np.arange(buffer_size),
                                size = batch_size,
                                replace = False)
        
        return [self.buffer[i] for i in index]
    

Here we will **deal with empty memory problem**: we pre-populate the memory by taking random actions and storing the experience (state, action, reward, nex_state)

In [12]:
# Instantiate memory
memory = Memory(max_size = memory_size)
for i in range(pretrain_length):
    # If it's the first step
    if i == 0:
        state = env.reset()
        
        state, stacked_frames = stack_frames(stacked_frames, state, True)
        
    # Get the next_state, the rewards, done by taking a random action
    choice = random.randint(1,len(possible_actions))-1
    action = possible_actions[choice]
    next_state, reward, done, _ = env.step(action)
    
    #env.render()
    
    # Stack the frames
    next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)
    
    
    # If the episode is finished (we're dead 3x)
    if done:
        # We finished the episode
        next_state = np.zeros(state.shape)
        
        # Add experience to memory
        memory.add((state, action, reward, next_state, done))
        
        # Start a new episode
        state = env.reset()
        
        # Stack the frames
        state, stacked_frames = stack_frames(stacked_frames, state, True)
        
    else:
        # Add experience to memory
        memory.add((state, action, reward, next_state, done))
        
        # Our new state is now the next_state
        state = next_state

### Tensorboard
To launch tensorboard: tensorboard --logdir=/tensorboard/dqn/1

In [13]:
# Setup TensorBoard Writer
writer = tf.summary.FileWriter("./tensorboard/dqn/1")

## Losses
tf.summary.scalar("Loss", DQNetwork.loss)

write_op = tf.summary.merge_all()

In [14]:
! --logdir="/tensorboard/dqn/1"

/bin/sh: --logdir=/tensorboard/dqn/1: No such file or directory


### Train the agent

Algorithm:

- Initialize weigths
- Init the environment
- Initialize the decay rate (epsilon greedy)


> **For episode to max_episode do:**
- Make new episode
- Set step to 0
- Observe the first state $s_0$

>> **While step < max_steps do:**
- Increase decay_rate
- With $e$ select a random action $a_t$, otherwise select $a_t = argmax_a Q(s_t,a)$
- Execute action $a_t$ in simulator and observe reward $r_{t+1}$ and new state $s{t+1}$
- Store transition $ @ $
- Sample random mini-batch from $D:@ $
- Set $ Q' = r $ if the episode ends alt $+1$, otherwise set $Q' = r + ymax_a Q(s',a') $
- Make a gradient descent step with loss $(Q'-Q(s,a))^2$

>> **endfor**

> **endfor**








  



In [15]:
#Function that predicts the action to take, random or argmax_aQ(s',a')

def predict_action(explore_start, explore_stop, decay_rate, decay_step, state, actions):
    ## EPSILON GREEDY
    # Choose action a from state s using epsilon greedy.
    ## First we randomize a number
    exp_exp_tradeoff = np.random.rand()

    # Here we'll use an improved version of our epsilon greedy strategy used in Q-learning notebook
    explore_probability = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * decay_step)
    
    if (explore_probability > exp_exp_tradeoff):
        # Make a random action (exploration)
        choice = random.randint(1,len(possible_actions))-1
        action = possible_actions[choice]
        
    else:
        # Get action from Q-network (exploitation)
        # Estimate the Qs values state
        Qs = sess.run(DQNetwork.output, feed_dict = {DQNetwork.inputs_: state.reshape((1, *state.shape))})
        
        # Take the biggest Q value (= the best action)
        choice = np.argmax(Qs)%3
        action = possible_actions[choice]
                
                
    return action, explore_probability

In [16]:
#from google.colab import files

# Saver will help us to save our model
saver = tf.train.Saver()

if training == True:
  with tf.Session() as sess:
      # Initialize the variables
      sess.run(tf.global_variables_initializer())

      # Initialize the decay rate (that will use to reduce epsilon) 
      decay_step = 0
      rewards_list = []

      for episode in range(total_episodes):
          # Set step to 0
          step = 0
          print(episode)

          # Initialize the rewards of the episode
          episode_rewards = []

          # Make a new episode and observe the first state
          state = env.reset()

          # Remember that stack frame function also call our preprocess function.
          state, stacked_frames = stack_frames(stacked_frames, state, True)

          while step < max_steps:
              step += 1

              #Increase decay_step
              decay_step +=1

              # Predict the action to take and take it
              action, explore_probability = predict_action(explore_start, explore_stop, decay_rate, decay_step, state, possible_actions)

              #Perform the action and get the next_state, reward, and done information
              next_state, reward, done, _ = env.step(action)
                
              reward -= 1

              #if episode_render:
              env.render()

              # Add the reward to total reward
              episode_rewards.append(reward)

              # If the game is finished
              if done:
                  # The episode ends so no next state
                  next_state = np.zeros((110,84), dtype=np.int)

                  next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)

                  # Set step = max_steps to end the episode
                  step = max_steps

                  # Get the total reward of the episode
                  total_reward = np.sum(episode_rewards)

                  print('Episode: {}'.format(episode),
                                'Total reward: {}'.format(total_reward),
                                'Explore P: {:.4f}'.format(explore_probability),
                              'Training Loss {:.4f}'.format(loss))

                  rewards_list.append((episode, total_reward))

                  # Store transition <st,at,rt+1,st+1> in memory D
                  memory.add((state, action, reward, next_state, done))

              else:
                  # Stack the frame of the next_state
                  next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)

                  # Add experience to memory
                  memory.add((state, action, reward, next_state, done))

                  # st+1 is now our current state
                  state = next_state


              ### LEARNING PART            
              # Obtain random mini-batch from memory
              batch = memory.sample(batch_size)
              states_mb = np.array([each[0] for each in batch], ndmin=3)
            
              actions_mb = np.array([each[1] for each in batch])
              rewards_mb = np.array([each[2] for each in batch]) 
              next_states_mb = np.array([each[3] for each in batch], ndmin=3)
              dones_mb = np.array([each[4] for each in batch])

              target_Qs_batch = []

              # Get Q values for next_state 
              Qs_next_state = sess.run(DQNetwork.output, feed_dict = {DQNetwork.inputs_: next_states_mb})

              # Set Q_target = r if the episode ends at s+1, otherwise set Q_target = r + gamma*maxQ(s', a')
              for i in range(0, len(batch)):
                  terminal = dones_mb[i]

                  # If we are in a terminal state, only equals reward
                  if terminal:
                      target_Qs_batch.append(rewards_mb[i])

                  else:
                      target = rewards_mb[i] + gamma * np.max(Qs_next_state[i])
                      target_Qs_batch.append(target)


              targets_mb = np.array([each for each in target_Qs_batch])

              loss, _ = sess.run([DQNetwork.loss, DQNetwork.optimizer],
                                      feed_dict={DQNetwork.inputs_: states_mb,
                                                 DQNetwork.target_Q: targets_mb,
                                                 DQNetwork.actions_: actions_mb})

              # Write TF Summaries
              summary = sess.run(write_op, feed_dict={DQNetwork.inputs_: states_mb,
                                                     DQNetwork.target_Q: targets_mb,
                                                     DQNetwork.actions_: actions_mb})
              writer.add_summary(summary, episode)
              writer.flush()

          # Save model every 5 episodes
          if episode % 2 == 0:
              save_path = saver.save(sess, "./models/model.ckpt")
              #files.download("./models/model.ckpt")
              print("Model Saved")
        
          

### Test and watch the agent

In [17]:
#For replay the game
#env = retro.make(game='SpaceInvaders-Atari2600')

with tf.Session() as sess:
    total_test_rewards = []
    
    # Load the model
    saver.restore(sess, "./models/9/model.ckpt")
    
    for episode in range(2):
        total_rewards = 0
        
        state = env.reset()
        state, stacked_frames = stack_frames(stacked_frames, state, True)
        
        print("****************************************************")
        print("EPISODE ", episode)
        
        while True:
            # Reshape the state
            state = state.reshape((1, *state_size))
            # Get action from Q-network 
            # Estimate the Qs values state
            Qs = sess.run(DQNetwork.output, feed_dict = {DQNetwork.inputs_: state})
            
            # Take the biggest Q value (= the best action)
            choice = np.argmax(Qs)%3
            action = possible_actions[choice]
            
            #Perform the action and get the next_state, reward, and done information
            next_state, reward, done, _ = env.step(action)
            env.render()
            
            total_rewards += reward

            if done:
                print ("Score", total_rewards)
                total_test_rewards.append(total_rewards)
                break
                
                
            next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)
            state = next_state
            
    env.render(close=True)
    env.close()

INFO:tensorflow:Restoring parameters from ./models/9/model.ckpt
****************************************************
EPISODE  0
Score 240.0
****************************************************
EPISODE  1
Score 240.0


### Probando los movimientos

In [69]:
#env = retro.make(game='SpaceInvaders-Atari2600')

with tf.Session() as sess:
    total_test_rewards = []
    
    # Load the model
    saver.restore(sess, "./models/7/model.ckpt")
    
    for episode in range(1):
        total_rewards = 0
        
        state = env.reset()
        
        print("****************************************************")
        print("EPISODE ", episode)
        move = 200
        while True:
            
            if move > 0:
                action = possible_actions[7]
            else: 
                action = possible_actions[6]
            move -=1
            
            next_state, reward, done, _ = env.step(action)
            env.render()
            
            total_rewards += reward

            if done:
                print ("Score", total_rewards)
                total_test_rewards.append(total_rewards)
                break
                
                
            state = next_state
            
    env.render(close=True)
    env.close()
#Action 0: Shoot
#Action 6: Move left
#Action 7: Move rigth

INFO:tensorflow:Restoring parameters from ./models/7/model.ckpt
****************************************************
EPISODE  0


IndexError: index 7 is out of bounds for axis 0 with size 3

In [18]:
env.render(close=True)

env.close()

In [116]:
#NES GAMES:
#Gradius: naves -
#Open golf tournament -
#Ice climbers - 
#BatmanReturns -
#DoubleDragon-Nes -

#SNES GAMES:
#Sunset riders - genesis
#IMGInternationalTourTennis - genesis
#Mortal kombat - genesis

import retro
for game in retro.data.list_games():
    print(game, retro.data.list_states(game))

1942-Nes ['1Player.Level1']
1943-Nes ['Level1']
3NinjasKickBack-Genesis ['1Player.Colt.Level1']
8Eyes-Nes ['1Player.Arabia.Level1']
AaahhRealMonsters-Genesis ['Level1']
AbadoxTheDeadlyInnerWar-Nes ['Level1']
AcceleBrid-Snes ['SilverMarine.DefaultSettings.Level1']
ActRaiser2-Snes ['Level1']
ActionPachio-Snes ['Level1']
AddamsFamily-GameBoy ['Level1']
AddamsFamily-Genesis ['Level1']
AddamsFamily-Nes ['Level1']
AddamsFamily-Sms ['Level1']
AddamsFamily-Snes ['Level1']
AddamsFamilyPugsleysScavengerHunt-Nes ['Level1']
AddamsFamilyPugsleysScavengerHunt-Snes ['Level1']
AdvancedBusterhawkGleylancer-Genesis ['NormalMover.Stage1']
Adventure-Atari2600 ['Start']
AdventureIsland-GameBoy ['Level1']
AdventureIsland3-Nes ['Level1']
AdventureIslandII-Nes ['FernIsland.Level1']
AdventuresOfBatmanAndRobin-Genesis ['Level1.Batman']
AdventuresOfBayouBilly-Nes ['GameA.Level1']
AdventuresOfDinoRiki-Nes ['1Player.Level1']
AdventuresOfDrFranken-Snes ['Level1']
AdventuresOfKidKleets-Snes ['Level1.London']
Adventu

FishingDerby-Atari2600 ['Start', 'Start.2P']
FistOfTheNorthStar-Nes ['Level1']
Flicky-Genesis ['Bird.Level1', 'Bird.Level10', 'Bird.Level11', 'Bird.Level12', 'Bird.Level13', 'Bird.Level14', 'Bird.Level15', 'Bird.Level16', 'Bird.Level17', 'Bird.Level18', 'Bird.Level19', 'Bird.Level2', 'Bird.Level20', 'Bird.Level21', 'Bird.Level22', 'Bird.Level23', 'Bird.Level24', 'Bird.Level25', 'Bird.Level26', 'Bird.Level27', 'Bird.Level28', 'Bird.Level29', 'Bird.Level3', 'Bird.Level30', 'Bird.Level31', 'Bird.Level32', 'Bird.Level33', 'Bird.Level34', 'Bird.Level35', 'Bird.Level36', 'Bird.Level37', 'Bird.Level38', 'Bird.Level39', 'Bird.Level4', 'Bird.Level40', 'Bird.Level41', 'Bird.Level42', 'Bird.Level43', 'Bird.Level44', 'Bird.Level45', 'Bird.Level46', 'Bird.Level47', 'Bird.Level48', 'Bird.Level5', 'Bird.Level6', 'Bird.Level7', 'Bird.Level8', 'Bird.Level9']
Flintstones-Genesis ['DefaultSettings.Level1']
Flintstones-Snes ['Level1']
FlintstonesTheRescueOfDinoAndHoppy-Nes ['Level1']
FlyingDragonTheSecret

MysteryQuest-Nes ['Level1']
NARC-Nes ['Level1']
NameThisGame-Atari2600 ['Start']
NewZealandStory-Genesis ['Level1']
NewZealandStory-Sms ['Level1']
Ninja-Sms ['Level1']
NinjaCrusaders-Nes ['Level1']
NinjaGaiden-Nes ['Level1']
NinjaGaiden-Sms ['Level1']
NinjaGaidenIIITheAncientShipOfDoom-Nes ['Level1']
NinjaGaidenIITheDarkSwordOfChaos-Nes ['Level1']
NinjaKid-Nes ['Level1']
NoahsArk-Nes ['Level1']
NormysBeachBabeORama-Genesis ['Level1']
OperationWolf-Nes ['Level1.CommunicationsCenter']
Ottifants-Genesis ['Level1']
Ottifants-Sms ['Level1']
OutToLunch-Snes ['Level1']
OverHorizon-Nes ['Level1']
POWPrisonersOfWar-Nes ['Level1']
PacInTime-Snes ['1Player.Level1']
PacManNamco-Nes ['Level1']
PacMania-Genesis ['Level1.BlockTown']
PacMania-Sms ['Level1.Blocktown']
PanicRestaurant-Nes ['Level1']
Paperboy-Genesis ['Level1.EasyStreet']
Paperboy-Nes ['Level1']
Paperboy-Sms ['Level1']
Paperboy2-Genesis ['Level1.EasyStreet']
Parodius-Nes ['1Player.PowerOption1.Auto.Easy']
Parodius-Snes ['1Player.VicViper

ViceProjectDoom-Nes ['Level1']
VideoPinball-Atari2600 ['Start']
Viewpoint-Genesis ['1Player.Level1']
Vigilante-Sms ['Level1']
VolguardII-Nes ['Level1']
WaniWaniWorld-Genesis ['Level1']
Wardner-Genesis ['Level1']
Warpman-Nes ['Level1']
WaynesWorld-Nes ['Level1']
WereBackADinosaursStory-Snes ['Level1']
WhipRush-Genesis ['Stage1']
Widget-Nes ['Level1']
WizardOfWor-Atari2600 ['Start']
WizardsAndWarriors-Nes ['Level1']
WiznLiz-Genesis ['Level1']
Wolfchild-Genesis ['Level1']
Wolfchild-Sms ['Level1']
Wolfchild-Snes ['Level1']
WolverineAdamantiumRage-Genesis ['Level1']
WonderBoyInMonsterWorld-Sms ['Level1']
WorldHeroes-Genesis ['Level1.Normal.HanzouVsJanna']
WrathOfTheBlackManta-Nes ['Level1']
WreckingCrew-Nes ['Level1']
XDRXDazedlyRay-Genesis ['Round1']
XKaliber2097-Snes ['Level1']
XMenMojoWorld-Sms ['Level1']
Xenon2Megablast-Genesis ['1Player.Easy.Level1']
Xenophobe-Nes ['Level1.DuckCharacter']
XeviousTheAvenger-Nes ['1Player.Level1']
Xexyz-Nes ['Level1']
YarsRevenge-Atari2600 ['Start']
Youk